This notebook contains the entiretly of the process of pre-processing, which includes the archi.. used form Data uplodation to storing to out weaviate vector database

In [10]:
import os
from pathlib import Path
import PyPDF2
from dotenv import load_dotenv
from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Load environment variables from .env file
load_dotenv()

/home/epein5/Desktop/ADVANCED-RAG/.venv/lib/python3.12/site-packages/PyPDF2/__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(


/home/epein5/Desktop/ADVANCED-RAG/.venv/lib/python3.12/site-packages/PyPDF2/__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(


True

In [11]:
def load_data(file_path: str) -> tuple:
    """
    Load data from TXT or PDF files with page tracking.
    
    Args:
        file_path: Path to the TXT or PDF file
        
    Returns:
        tuple: (full_text, list of (text, page_num) tuples)
    """
    file_path = Path(file_path)
    
    if not file_path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")
    
    if file_path.suffix.lower() == '.txt':
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            return text, [(text, 1)]
    
    elif file_path.suffix.lower() == '.pdf':
        full_text = ""
        pages_data = []
        
        with open(file_path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            for page_num, page in enumerate(pdf_reader.pages, start=1):
                page_text = page.extract_text()
                full_text += page_text
                pages_data.append((page_text, page_num))
        
        return full_text, pages_data
    
    else:
        raise ValueError(f"Unsupported file type: {file_path.suffix}")

In [12]:
def clean_text(text: str) -> str:
    """
    Clean text by removing extra whitespace, newlines, and normalizing spaces.
    
    Args:
        text: Raw text to clean
        
    Returns:
        str: Cleaned text
    """
    # Replace multiple newlines with a single newline
    text = '\n'.join(line.strip() for line in text.split('\n') if line.strip())
    
    # Replace multiple spaces with a single space
    text = ' '.join(text.split())
    
    # Optional: Replace newlines with spaces for more compact chunks
    # text = text.replace('\n', ' ')
    
    return text

In [13]:
loaded_data, pages_data = load_data("../dummy_data/nepal_constiution.pdf")

# Build character position to page number mapping BEFORE any truncation or cleaning
# This ensures we can map chunk positions back to original PDF pages
char_to_page = {}
current_pos = 0
for page_text, page_num in pages_data:
    for i in range(len(page_text)):
        char_to_page[current_pos + i] = page_num
    current_pos += len(page_text)

# Now truncate and clean the data
loaded_data = loaded_data
loaded_data = clean_text(loaded_data)

print(f"Loaded {len(loaded_data)} characters")
print(f"Mapped {len(char_to_page)} character positions to pages")

Loaded 334378 characters
Mapped 343982 character positions to pages


In [ ]:
import re

def recursive_splitter(text, max_chunk_size=1500, separators=None):
    """
    Splits text hierarchically without overlaps.
    Returns: List of chunks (text) and List of span annotations (start, end).
    """
    if separators is None:
        # Hierarchy: Double Newline -> Single Newline -> Sentence End -> Space -> Character
        separators = ["\n\n", "\n", r"(?<=[.!?])\s+", " ", ""]

    def split_recursive(input_text, current_separators, offset=0):
        # Base Case: Text is small enough or no more separators to try
        if len(input_text) <= max_chunk_size or not current_separators:
            return [input_text], [(offset, offset + len(input_text))]

        # Select the highest priority separator
        sep = current_separators[0]
        remaining_seps = current_separators[1:]

        # Split the text by the separator
        if sep == "": # Final fallback to character-by-character if needed
            raw_splits = list(input_text)
        elif sep.startswith("(?<="): # Regex for sentence boundaries
            raw_splits = re.split(sep, input_text)
        else:
            raw_splits = input_text.split(sep)

        final_chunks = []
        final_spans = []
        
        current_buffer = ""
        current_offset = offset

        for part in raw_splits:
            # If we used a separator, re-add it (except for the last part)
            # This ensures we don't lose punctuation or newlines
            if sep != "" and not sep.startswith("(?<=") and part != raw_splits[-1]:
                part += sep
            
            # Check if this single part is ALREADY too big
            if len(part) > max_chunk_size:
                # If we have a buffer, flush it first
                if current_buffer:
                    final_chunks.append(current_buffer)
                    final_spans.append((current_offset, current_offset + len(current_buffer)))
                    current_offset += len(current_buffer)
                    current_buffer = ""
                
                # Recursively split the "too big" part using the NEXT separator
                sub_chunks, sub_spans = split_recursive(part, remaining_seps, current_offset)
                final_chunks.extend(sub_chunks)
                final_spans.extend(sub_spans)
                current_offset = sub_spans[-1][1]
            
            # If adding this part exceeds max_size, flush the buffer
            elif len(current_buffer) + len(part) > max_chunk_size:
                final_chunks.append(current_buffer)
                final_spans.append((current_offset, current_offset + len(current_buffer)))
                current_offset += len(current_buffer)
                current_buffer = part
            
            else:
                current_buffer += part

        # Final flush for whatever is left in the buffer
        if current_buffer:
            final_chunks.append(current_buffer)
            final_spans.append((current_offset, current_offset + len(current_buffer)))

        return final_chunks, final_spans

    return split_recursive(text, separators)

In [6]:
chunks = recursive_splitter(loaded_data)

In [7]:
def get_chunk_page(span_start, char_to_page):
    """
    Get the original PDF page number for a chunk based on its starting position.
    
    Args:
        span_start: Starting character position of the chunk in the document
        char_to_page: Dictionary mapping character positions to original PDF page numbers
        
    Returns:
        int: Original PDF page number (1-indexed)
    """
    # Look up the exact position first
    if span_start in char_to_page:
        return char_to_page[span_start]
    
    # If exact position not found, find the nearest position before span_start
    positions = sorted(char_to_page.keys())
    for pos in reversed(positions):
        if pos < span_start:
            return char_to_page[pos]
    
    # Default to page 1 if position is before all mapped positions
    return 1

## Contextual Retrieval with Gemini + Prompt Caching

Generate concise context for each chunk using Google Gemini with prompt caching enabled.
This explains where each chunk sits within the whole document to improve retrieval accuracy.


In [8]:
from pydantic import BaseModel, Field

class ChunkResponse(BaseModel):
    context: str = Field(description="Concise context explaining where this chunk sits within the document.")
    chunk_type: str = Field(description="Classification or type of the chunk (e.g., 'section', 'definition').")
    breadcrumb: str = Field(description="Hierarchical path or breadcrumb showing chunk location in document structure.")

In [9]:
import google.genai as genai
from tqdm import tqdm
import json
from datetime import timedelta


def generate_context_for_chunk(client, cache_name, chunk_text, chunk_idx=None, model_name="gemini-2.5-flash"):
    """
    Generate context for a single chunk using Gemini with prompt caching.
    
    Args:
        client: Google Generative AI client
        cache_name: Name of the cached content
        chunk_text: Individual chunk text
        chunk_idx: Optional chunk index for error messages
        model_name: Model to use
        
    Returns:
        ChunkResponse: Pydantic model with context, chunk_type, and breadcrumb
    """
    chunk_prompt = f"""Here is the chunk to analyze:
<chunk>
{chunk_text}
</chunk>

Provide context for this chunk."""
    
    idx_str = f"Chunk {chunk_idx}: " if chunk_idx is not None else ""
    
    try:
        response = client.models.generate_content(
            model=model_name,
            contents=chunk_prompt,
            config=genai.types.GenerateContentConfig(
                cached_content=cache_name,
                response_mime_type="application/json",
                response_schema=ChunkResponse.model_json_schema(),
                temperature=0.3,
                max_output_tokens=2048,
                safety_settings=[
                    genai.types.SafetySetting(
                        category="HARM_CATEGORY_DANGEROUS_CONTENT",
                        threshold="BLOCK_NONE"
                    )
                ],
            )
        )
        
        # Extract cache metrics
        cache_read_tokens = getattr(response.usage_metadata, 'cache_read_input_tokens', 0) or 0
        
        # Check if response was truncated
        if hasattr(response, 'candidates') and response.candidates:
            finish_reason = getattr(response.candidates[0], 'finish_reason', None)
            if finish_reason and str(finish_reason) == 'MAX_TOKENS':
                print(f"{idx_str}Response truncated")
                return None
        
        # Parse JSON response
        if response.text:
            text = response.text.strip()
            if text.startswith('```json'):
                text = text[7:]
            if text.startswith('```'):
                text = text[3:]
            if text.endswith('```'):
                text = text[:-3]
            text = text.strip()
            
            data = json.loads(text)
            result = ChunkResponse(**data)
            result._cache_read_tokens = cache_read_tokens
            return result
        
        return None
        
    except json.JSONDecodeError as e:
        print(f"{idx_str}JSON parse error: {str(e)[:50]}")
        return None
    except Exception as e:
        print(f"{idx_str}Error: {e}")
        return None

In [10]:
def create_chunk_data(idx, chunk_text, llm_context_response, start, end, page_num):
    """
    Create a chunk data dictionary with page information.
    
    Args:
        idx: Chunk index
        chunk_text: Original chunk text
        context: ChunkResponse object with context, chunk_type, and breadcrumb
        start: Span start
        end: Span end
        page_num: Page number
        
    Returns:
        dict: Chunk data with context and page info
    """
    # Extract fields from ChunkResponse object
    context_text = llm_context_response.context if llm_context_response else ""
    chunk_type = llm_context_response.chunk_type if llm_context_response else ""
    breadcrumb = llm_context_response.breadcrumb if llm_context_response else ""
    
    contextualized_chunk = f"{context_text}\n\n{chunk_text}" if context_text else chunk_text
    
    return {
        "index": idx,
        "page": page_num,
        "original_chunk": chunk_text,
        "context": context_text,
        "chunk_type": chunk_type,
        "breadcrumb": breadcrumb,
        "contextualized_chunk": contextualized_chunk,
        "span": (start, end)
    }

In [11]:
def generate_chunk_contexts(
    document_text, 
    chunks_with_spans, 
    char_to_page, 
    model_name="gemini-2.5-flash",
    cache_ttl_hours=1
):
    """
    Generate context for all chunks using prompt caching.
    
    Args:
        document_text: The full document text
        chunks_with_spans: Tuple of (chunks_list, spans_list)
        char_to_page: Dictionary mapping character positions to PDF page numbers
        model_name: Google Generative AI model to use
        cache_ttl_hours: Cache time-to-live in hours
        
    Returns:
        List of dicts with chunk data and context
    """
    chunks_list, spans_list = chunks_with_spans
    client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))
    chunk_data = []
    
    system_instruction = """Generate retrieval context for the following chunk. The context should:
- Be 2-3 sentences that situate this chunk within the document
- Include key terms/topics from surrounding sections
- Explain the chunk's purpose or what question it answers
- Be self-contained (assume the reader only sees the chunk + your context)"""
    
    # Create cache with document + system instructions
    print("Creating prompt cache...")
    try:
        cache_doc_text = f"<document>\n{document_text}\n</document>\n\nI will ask you to analyze chunks from this document."
        
        cache = client.caches.create(
            model=model_name,
            config=genai.types.CreateCachedContentConfig(
                display_name="document_context_cache",
                system_instruction=system_instruction,
                contents=[cache_doc_text],
                ttl=f"{int(cache_ttl_hours * 3600)}s"
            )
        )
        
        cache_name = cache.name
        cache_tokens = getattr(cache.usage_metadata, 'total_token_count', 0) or 0
        print(f"Cache created: {cache_tokens:,} tokens cached")
        
    except Exception as e:
        print(f"Error creating cache: {e}")
        raise
    
    # Process chunks
    print(f"Processing {len(chunks_list)} chunks...")
    total_cache_reads = 0
    failed_count = 0
    
    for idx, (chunk_text, (start, end)) in enumerate(
        tqdm(zip(chunks_list, spans_list), total=len(chunks_list), desc="Chunks")
    ):
        try:
            response = generate_context_for_chunk(
                client, cache_name, chunk_text, chunk_idx=idx, model_name=model_name
            )
            
            if response is None:
                failed_count += 1
                continue
            
            page_num = get_chunk_page(start, char_to_page)
            chunk_item = create_chunk_data(idx, chunk_text, response, start, end, page_num)
            chunk_data.append(chunk_item)
            
            # Track cache usage
            cache_reads = getattr(response, '_cache_read_tokens', 0)
            total_cache_reads += cache_reads
            
        except Exception as e:
            print(f"Chunk {idx}: Error - {e}")
            failed_count += 1
            continue
    
    # Summary
    print(f"\nProcessed: {len(chunk_data)}/{len(chunks_list)} chunks")
    if total_cache_reads > 0:
        print(f"Cache reads: {total_cache_reads:,} tokens")
    if failed_count > 0:
        print(f"Failed: {failed_count} chunks")
    
    # Cleanup
    try:
        client.caches.delete(name=cache_name)
        print("Cache cleaned up")
    except Exception as e:
        print(f"Warning: Could not delete cache: {e}")
    
    return chunk_data

In [9]:
# Generate contexts for all chunks with original PDF page numbers
chunk_contexts = generate_chunk_contexts(loaded_data, chunks, char_to_page)

NameError: name 'generate_chunk_contexts' is not defined

In [7]:
# Display sample chunk
print(f"Total chunks: {len(chunk_contexts)}")

if chunk_contexts:
    sample = chunk_contexts[0]
    print(f"\nSample (Index 0):")
    print(f"Original: {sample['original_chunk'][:200]}...")
    print(f"Page: {sample['page']}, Type: {sample['chunk_type']}, Path: {sample['breadcrumb']}")
    print(f"Context: {sample['context']}")
    print(f"\nContextualized: {sample['contextualized_chunk'][:300]}...")

print("\nReady for Weaviate upload")

NameError: name 'chunk_contexts' is not defined

In [14]:
# chunk_contexts

## Use OpenAI's embeddings to make embeddings

Generate enbeddings out of contextualized_chunks

In [15]:
from openai import AzureOpenAI
from tqdm import tqdm

# Azure OpenAI Embeddings Configuration
endpoint = "https://grow-me82mm7z-eastus2.cognitiveservices.azure.com/"
deployment = "text-embedding-3-large"

embedding_client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=endpoint,
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

def generate_embeddings(texts: list[str], batch_size: int = 20) -> list[list[float]]:
    """
    Generate embeddings for a list of texts using Azure OpenAI.
    
    Args:
        texts: List of text strings to embed
        batch_size: Number of texts to embed per API call
        
    Returns:
        List of embedding vectors
    """
    all_embeddings = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
        batch = texts[i:i + batch_size]
        response = embedding_client.embeddings.create(
            input=batch,
            model=deployment
        )
        # Sort by index to maintain order
        batch_embeddings = [item.embedding for item in sorted(response.data, key=lambda x: x.index)]
        all_embeddings.extend(batch_embeddings)
    
    return all_embeddings

# Generate embeddings for all contextualized chunks
contextualized_texts = [chunk["contextualized_chunk"] for chunk in chunk_contexts]
embeddings = generate_embeddings(contextualized_texts)

# Add embeddings to chunk_contexts
for chunk, embedding in zip(chunk_contexts, embeddings):
    chunk["vector"] = embedding

print(f"Generated {len(embeddings)} embeddings")
print(f"Embedding dimension: {len(embeddings[0])}")

Generating embeddings: 100%|██████████| 13/13 [00:19<00:00,  1.48s/it]

Generated 252 embeddings
Embedding dimension: 3072


## Prepare Data for Weaviate Upload

Convert the contextualized chunks to Weaviate-ready format with embeddings and metadata.


In [1]:
import weaviate
from datetime import datetime, timezone

# Connect to Weaviate
client = weaviate.connect_to_local(host="localhost", port=8080)
print(f"Connected to Weaviate: {client.is_ready()}")

Connected to Weaviate: True


In [ ]:
# # Delete existing collection if it exists (fresh start)
# try:
#     client.collections.delete("Document")
#     print("Deleted existing Document collection")
# except Exception as e:
#     print(f"No existing collection: {e}")

Deleted existing Document collection


In [18]:
# Create Document collection schema
schema = {
    "class": "Document",
    "vectorizer": "none",  # We provide our own vectors
    "properties": [
        {"name": "content", "dataType": ["text"], "description": "Original chunk text"},
        {"name": "context", "dataType": ["text"], "description": "LLM-generated context"},
        {"name": "contextualized_chunk", "dataType": ["text"], "description": "Context + content combined"},
        {"name": "chunk_type", "dataType": ["string"], "description": "Type of chunk"},
        {"name": "breadcrumb", "dataType": ["string"], "description": "Document path"},
        {"name": "source", "dataType": ["string"], "description": "Source file"},
        {"name": "chunk_id", "dataType": ["int"], "description": "Chunk index"},
        {"name": "page_number", "dataType": ["int"], "description": "PDF page number"},
        {"name": "created_at", "dataType": ["date"], "description": "Upload timestamp"},
    ]
}

client.collections.create_from_dict(schema)
print("Document collection created")

Document collection created


In [19]:
# Upload chunks with vectors to Weaviate
documents = client.collections.get("Document")
current_time = datetime.now(timezone.utc).isoformat()

for chunk in tqdm(chunk_contexts, desc="Uploading to Weaviate"):
    documents.data.insert(
        properties={
            "content": chunk["original_chunk"],
            "context": chunk["context"],
            "contextualized_chunk": chunk["contextualized_chunk"],
            "chunk_type": chunk["chunk_type"],
            "breadcrumb": chunk["breadcrumb"],
            "source": "nepal_constitution.pdf",
            "chunk_id": chunk["index"],
            "page_number": chunk["page"],
            "created_at": current_time,
        },
        vector=chunk["vector"]
    )

print(f"Uploaded {len(chunk_contexts)} chunks to Weaviate")

Uploading to Weaviate: 100%|██████████| 252/252 [00:03<00:00, 72.27it/s]

Uploaded 252 chunks to Weaviate


In [6]:
# Verify upload - check total count and sample documents
documents = client.collections.get("Document")

# Get actual count using aggregate
count = documents.aggregate.over_all(total_count=True).total_count
print(f"Total documents in Weaviate: {count}\n")

# Fetch samples WITH vectors
response = documents.query.fetch_objects(limit=3, include_vector=True)
print("Sample documents:")
for obj in response.objects:
    props = obj.properties
    print(f"Chunk {props['chunk_id']} (Page {props['page_number']}):")
    print(f"  Type: {props['chunk_type']}")
    print(f"  Path: {props['breadcrumb']}")
    
    # Vector handling
    if obj.vector:
        vector_list = list(obj.vector) if not isinstance(obj.vector, list) else obj.vector
        print(f"  Vector dimension: {len(vector_list)}")
        print(f"  Vector (first 5 values): {vector_list[:5]}")
    else:
        print(f"  Vector: Not included")
    print()

Total documents in Weaviate: 252

Sample documents:
Chunk 203 (Page 119):
  Type: article section
  Path: PART 27 Other Commissions > 259. Functions, duties and powers of National Inclusion Commission
  Vector dimension: 1
  Vector (first 5 values): ['default']

Chunk 91 (Page 55):
  Type: section
  Path: PART -11 Judiciary > 128. Supreme Court > 129. Appointment and qualifications of Chief Justice and Justices of the Supreme Court
  Vector dimension: 1
  Vector (first 5 values): ['default']

Chunk 196 (Page 116):
  Type: section
  Path: PART 27 Other Commissions > 256. Functions, duties and powers of National Dalit Commission
  Vector dimension: 1
  Vector (first 5 values): ['default']



In [ ]:
# # Delete all collections from Weaviate database
# collections = client.collections.list_all()
# for collection_name in collections:
#     try:
#         client.collections.delete(collection_name)
#         print(f"Deleted collection: {collection_name}")
#     except Exception as e:
#         print(f"Error deleting {collection_name}: {e}")

# print("All collections deleted. Database is now empty.")

Deleted collection: Document
All collections deleted. Database is now empty.
